In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


LeNet 5 Architecture:-

Contain 7 Layers

Input (32 x 32 x 1)

Convolution Layer C1
SubPooling Layer S2
Covolution Layer C3
SubSampling Layer S4
FullyConnected Layer Convolutions(Flattened Layer) C5
Fully Connected Layer F6
Output Layer Fully Connected 


Importing the Dependencies

In [2]:
import numpy as np
import pandas as pd

Loading the DataSet


In [3]:
df_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [4]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = df_train.iloc[:,1:].values.reshape(-1,28,28)/255.0
y_train = df_train.iloc[:, 0].values
X_test = df_test.values.reshape(-1, 28, 28) / 255.0
# Pad to 32x32 as in original LeNet
X_train = np.pad(X_train, ((0,0),(2,2),(2,2)), 'constant')
X_test = np.pad(X_test, ((0,0),(2,2),(2,2)), 'constant')

In [7]:
X_train.shape

(42000, 32, 32)

In [8]:
y_train.shape

(42000,)

Basic Layers Implementation

In [9]:
# Activation and Loss Functions

def relu(x):
    return np.maximum(0, x)

def relu_backward(dout, x):
    dx = dout.copy()
    dx[x <= 0] = 0
    return dx

def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)

def cross_entropy_loss(pred, label):
    loss = -np.log(pred[label] + 1e-12)
    d_out = pred.copy()
    d_out[label] -= 1
    return loss, d_out


In [10]:
# Convolution and Pooling Layers

def conv2d(img, kernel):
    k = kernel.shape[0]
    out_dim = img.shape[0] - k + 1
    out = np.zeros((out_dim, out_dim))
    for i in range(out_dim):
        for j in range(out_dim):
            out[i,j] = np.sum(img[i:i+k, j:j+k] * kernel)
    return out

def conv2d_backward(d_out, img, kernel):
   
    k = kernel.shape[0]
    d_kernel = np.zeros_like(kernel)
    out_dim = d_out.shape[0]
    for i in range(k):
        for j in range(k):
            d_kernel[i,j] = np.sum(d_out * img[i:i+out_dim, j:j+out_dim])
  
    d_img = np.zeros_like(img)
    for i in range(out_dim):
        for j in range(out_dim):
            d_img[i:i+k, j:j+k] += d_out[i,j] * kernel
    return d_kernel, d_img

def avg_pool(img, size=2):
    s = size
    out_dim = img.shape[0] // s
    out = np.zeros((out_dim, out_dim))
    for i in range(out_dim):
        for j in range(out_dim):
            out[i,j] = np.mean(img[i*s:(i+1)*s, j*s:(j+1)*s])
    return out

def avg_pool_backward(d_out, img, size=2):
    s = size
    out_dim = d_out.shape[0]
    d_img = np.zeros_like(img)
    for i in range(out_dim):
        for j in range(out_dim):
            d_img[i*s:(i+1)*s, j*s:(j+1)*s] = d_out[i,j] / (s*s)
    return d_img

Simple Forward Pass

In [11]:
class SimpleLeNet:
    def __init__(self):
        
        self.k1 = [np.random.randn(5,5)*0.01 for _ in range(2)] 
        self.k3 = [np.random.randn(5,5)*0.01 for _ in range(4)]
       
        self.W1 = np.random.randn(30, 4*5*5)*0.01
        self.W2 = np.random.randn(20, 30)*0.01
        self.W3 = np.random.randn(10, 20)*0.01
        self.lr = 0.01

    def forward(self, x):
        self.x = x
       
        self.C1 = [relu(conv2d(x, k)) for k in self.k1]
       
        self.S2 = [avg_pool(m) for m in self.C1]
     
        self.C3 = [relu(conv2d(self.S2[i%2], self.k3[i])) for i in range(4)]
    
        self.S4 = [avg_pool(m) for m in self.C3]
       
        self.flat = np.concatenate([m.flatten() for m in self.S4])
        
        self.fc1 = relu(self.W1 @ self.flat)
        self.fc2 = relu(self.W2 @ self.fc1)
        self.out = softmax(self.W3 @ self.fc2)
        return self.out

    def backward(self, d_out):
        
        dW3 = np.outer(d_out, self.fc2)
        d_fc2 = self.W3.T @ d_out
        d_fc2 = relu_backward(d_fc2, self.fc2)
        dW2 = np.outer(d_fc2, self.fc1)
        d_fc1 = self.W2.T @ d_fc2
        d_fc1 = relu_backward(d_fc1, self.fc1)
        dW1 = np.outer(d_fc1, self.flat)

       
        d_flat = self.W1.T @ d_fc1
        S4_shapes = [m.shape for m in self.S4]
        dS4 = []
        idx = 0
        for shape in S4_shapes:
            size = shape[0]*shape[1]
            dS4.append(d_flat[idx:idx+size].reshape(shape))
            idx += size

        
        dC3 = [avg_pool_backward(dS4[i], self.C3[i]) for i in range(4)]

        
        dS2 = [np.zeros_like(self.S2[0]) for _ in range(2)]
        for i in range(4):
            dk3, d_input = conv2d_backward(dC3[i], self.S2[i%2], self.k3[i])
            self.k3[i] -= self.lr * dk3
            dS2[i%2] += d_input

       
        dC1 = [avg_pool_backward(dS2[i], self.C1[i]) for i in range(2)]

       
        for i in range(2):
            dk1, d_input = conv2d_backward(dC1[i], self.x, self.k1[i])
            self.k1[i] -= self.lr * dk1

        
        self.W3 -= self.lr * dW3
        self.W2 -= self.lr * dW2
        self.W1 -= self.lr * dW1

    def train_step(self, x, label):
        pred = self.forward(x)
        loss, d_out = cross_entropy_loss(pred, label)
        self.backward(d_out)
        return loss
